In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('word_count').getOrCreate()


21/10/14 07:31:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


1 Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [35]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df_1 = spark.read.csv('owid-covid-data.csv', header=True).select("iso_code", "date", "location", "total_cases", "population")

df_1\
.select('iso_code', 'location', ((df_1.total_cases/df_1.population)*100).alias('percent_of_sick'))\
.where(col('date') == '2021-03-31')\
.orderBy(col('percent_of_sick').desc())\
.show(15)

+--------+-------------+------------------+
|iso_code|     location|   percent_of_sick|
+--------+-------------+------------------+
|     AND|      Andorra|15.543907331909661|
|     MNE|   Montenegro|14.523725364693293|
|     CZE|      Czechia|14.308848404077997|
|     SMR|   San Marino|13.937179562732041|
|     SVN|     Slovenia|10.370805779121204|
|     LUX|   Luxembourg| 9.847342390123583|
|     ISR|       Israel| 9.625106044786802|
|     USA|United States| 9.203010995860707|
|     SRB|       Serbia| 8.826328557933492|
|     BHR|      Bahrain| 8.488860079114566|
|     PAN|       Panama| 8.228739065460761|
|     PRT|     Portugal| 8.058699735120369|
|     EST|      Estonia| 8.022681579659551|
|     SWE|       Sweden| 7.969744347858805|
|     LTU|    Lithuania| 7.938864728274825|
+--------+-------------+------------------+
only showing top 15 rows



2 Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [84]:
df_2 = spark.read.csv('owid-covid-data.csv', header=True, inferSchema=True).select("iso_code", "date", "location", "new_cases")

df_2\
.select('date', 'location', 'new_cases')\
.where((col('date') > '2021-03-24')\
       & (col('date') <= '2021-03-31')\
       & (~col('location').isin('North America', 'South America','World','Europe', 'European Union', 'Asia')))\
.orderBy(col('new_cases').desc()) \
.show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil| 100158.0|
|2021-03-31|       Brazil|  90638.0|
|2021-03-27|       Brazil|  85948.0|
|2021-03-30|       Brazil|  84494.0|
|2021-03-26|       Brazil|  84245.0|
|2021-03-26|United States|  77321.0|
|2021-03-31|        India|  72330.0|
|2021-03-29|United States|  69429.0|
|2021-03-28|        India|  68020.0|
|2021-03-25|United States|  67465.0|
+----------+-------------+---------+
only showing top 10 rows



3 Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [15]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window

df_3 = spark.read.csv('owid-covid-data.csv', header=True, inferSchema=True)\
.select("iso_code", col("date").cast(DateType()), "location", "new_cases")\
.where((col('location') == 'Russia'))

w = Window().partitionBy('location').orderBy('date')

df_3\
.select('date',\
        (lag('new_cases').over(w)).alias('previous_cases'),\
        'new_cases',\
        (df_3.new_cases - lag('new_cases').over(w)).alias('delta'))\
.where((col('date') > '2021-03-24') & (col('date') <= '2021-03-31'))\
.orderBy('date')\
.show()

+----------+--------------+---------+------+
|      date|previous_cases|new_cases| delta|
+----------+--------------+---------+------+
|2021-03-25|        8769.0|   9128.0| 359.0|
|2021-03-26|        9128.0|   9073.0| -55.0|
|2021-03-27|        9073.0|   8783.0|-290.0|
|2021-03-28|        8783.0|   8979.0| 196.0|
|2021-03-29|        8979.0|   8589.0|-390.0|
|2021-03-30|        8589.0|   8162.0|-427.0|
|2021-03-31|        8162.0|   8156.0|  -6.0|
+----------+--------------+---------+------+

